In [2]:
import cv2
import mediapipe as mp
import time

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Parameters
movement_threshold = 50  # Higher value = less sensitive
bad_sleep_counter = 0
total_frames = 0

# Start webcam
cap = cv2.VideoCapture(0)
print("🎥 Camera started. Monitoring sleep... (Press 'q' to quit)")

# Previous keypoint positions
prev_landmarks = None

start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame.")
        break

    # Flip frame for natural view
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        # Draw pose landmarks
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Get current landmark positions
        landmarks = [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]

        # Compare with previous landmarks
        if prev_landmarks:
            movement = sum(
                ((curr[0] - prev[0])**2 + (curr[1] - prev[1])**2 + (curr[2] - prev[2])**2)**0.5
                for curr, prev in zip(landmarks, prev_landmarks)
            )
            if movement > movement_threshold:
                bad_sleep_counter += 1
                cv2.putText(frame, "⚠️ Restless Sleep Detected", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        prev_landmarks = landmarks

    total_frames += 1

    # Display frame
    cv2.imshow("Sleep Monitor", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Results
duration = time.time() - start_time
print("\n📊 Sleep Monitoring Summary:")
print(f"🕒 Total Monitoring Time: {duration/60:.2f} minutes")
print(f"📸 Total Frames Analyzed: {total_frames}")
print(f"⚠️ Restless Frames Detected: {bad_sleep_counter}")

if bad_sleep_counter / total_frames > 0.1:
    print("\n💡 Suggestion: Your sleep was quite restless. Try relaxation techniques before bed.")
else:
    print("\n✅ Good sleep detected. Your sleep pattern seems healthy!")


🎥 Camera started. Monitoring sleep... (Press 'q' to quit)

📊 Sleep Monitoring Summary:
🕒 Total Monitoring Time: 0.48 minutes
📸 Total Frames Analyzed: 605
⚠️ Restless Frames Detected: 0

✅ Good sleep detected. Your sleep pattern seems healthy!
